In [52]:
%reload_ext autoreload
%autoreload 2
# BASE
import os
import json
# COOL
from pyeed.core import ProteinRecord

In [53]:
# read in the protein records from the json
current_path = os.path.dirname(os.getcwd())
read_blast_folder = os.path.join(current_path, "TEM-lactamase", "data", "data_fetch_list_ids_all")
# /home/niklas/Desktop/Job_Niklas/TEM-lactamase/data/data_blast_search_big_list001test_all_10
# list all the files in the directory
# read in the saved fetched results
protein_list = []
for file in os.listdir(read_blast_folder):
    with open(os.path.join(read_blast_folder, file), "r") as f:
        protein_list.append(ProteinRecord.from_json(f))

print('The fetched list of results is: ' +  str(len(protein_list)))

# now we check for double entries, we check the ids
count = 0
ids = []
filtered_protein_list = []

for protein in protein_list:
    if protein.id not in ids:
        ids.append(protein.id)
        filtered_protein_list.append(protein)
    else:
        count += 1

print('The number of double entries is: ' + str(count))
print('The number of filtered entries is: ' + str(len(filtered_protein_list)))

The fetched list of results is: 210
The number of double entries is: 0
The number of filtered entries is: 210


In [54]:
# read in the file from json
tem_dic = {}

with open(os.path.join(current_path, "TEM-lactamase", "TEM_Ids.json"), "r") as f:
    tem_dic = json.load(f)

# now read in the ids with the tem names and rewrite the list in a dic
filtered_protein_dic = {}
# filtered_protein_dic has the protein name like tem-1 as key and the protein record as value

for protein in filtered_protein_list:
    # check if one of the regions has a region with a start name "blastquery:"
    found_in_region = False
    for region in protein.regions:
        if region.name != None:
            if "blastquery" in region.name:
                found_in_region = True
                filtered_protein_dic[region.name.split(":")[1]] = {'protein': protein, 'alignment': ''}
            
    if not found_in_region:
        continue
        # filtered_protein_dic['-'+protein.id] = protein
            

print('The number of proteins in the filtered protein dic is: ' + str(len(filtered_protein_dic)))


The number of proteins in the filtered protein dic is: 210


In [58]:
# now we run clustal omega on the proteins
from pyeed.tools.clustalo import ClustalOmega

sequences = [protein['protein'].sequence for protein in filtered_protein_dic.values()]

count = sum(1 for a, b in zip(sequences[100], sequences[10]) if a != b)
print(count)

# fix the format for clustal omega
# sequences = [
#     ">seq1\nMTHKLLLTLLFTLLFSSAYSRG",
#     ">seq2\nMTHKILLLTLLFTLLFSSAYSRG",
#     ">seq3\nMTHKILLLTLLFTLLFSSCYSRG",
# ]
sequences_filtered = []

for i, seq in enumerate(sequences):
    sequences_filtered.append(">seq" + str(i) + "\n" + seq)


clustalo = ClustalOmega()
alignment = clustalo.align(sequences_filtered)

# now we need the alignment put in the dic for later use and export
for i, protein in enumerate(filtered_protein_dic.values()):
    protein['alignment'] = alignment[i].seq


8


In [ ]:
output_folder_blast_search = os.path.join(current_path, "TEM-lactamase", "data", "alignment_protein_ids_fetch_all")
os.makedirs(output_folder_blast_search, exist_ok=True)

def dumper(obj):
    try:
        return obj.json()
    except:
        return obj.__dict__


# now we save the dic with the alignment in json format
# save the blast search results
counter = 0
for key, value in filtered_protein_dic.items():
    with open(os.path.join(output_folder_blast_search, key + ".json"), "w") as f:
        json.dump(value, f, default=dumper)
        counter += 1